### AutoReplyAgent

`AutoReplyAgent` is a special agent in `AgentChat` that created on top of an arbitray `IAgent`. It contains an inner agent and an auto-reply function. When replying, `AutoReplyAgent` will first call the auto-reply function to generate a reply. If the auto-reply function returns a reply, the reply will be used as the reply of the `AutoReplyAgent`. Otherwise, the inner agent will be called to generate a reply.

`AutoReplyAgent` is useful when you want to have more control over how an agent replies under some circumstances. Or when you want to create a more general purpose agent that can reply to a wider range of messages on top of a list of task-specific agents.

In this notebook, we will first introduce the basic usage of `AutoReplyAgent`. We will first show how to create an `AutoReplyAgent` that always use auto-reply function to generate a reply. Then we will show how to create an `AutoReplyAgent` that only use auto-reply function under certain conditions and uses its inner agent otherwise. Finally, we will show how to combine multiple agents into a single `AutoReplyAgent` to accomplish a more general purpose.

In [3]:
// Install dependencies
#i "nuget:https://www.myget.org/F/agentchat/api/v3/index.json"

#r "nuget:AgentChat.OpenAI,*-*"
#r "nuget:AgentChat.Example.Share,*-*"
// this package provides FunctionDefinition for `dotnet-interactive` command
#r "nuget:AgentChat.DotnetInteractiveFunction,*-*"

Restore sources https://www.myget.org/F/agentchat/api/v3/index.json Installed Packages AgentChat.DotnetInteractiveFunction, 0.3.1-6712552592 AgentChat.Example.Share, 0.3.1-6712552592 AgentChat.OpenAI, 0.3.1-6712552592

### 1. Create an `AutoReplyAgent` that always use auto-reply function to generate a reply

To prevent the inner agent from generating a reply, we just need to make sure the auto-reply function always returns a reply.

In [18]:
using AgentChat;
using AgentChat.Example.Share;
using AgentChat.OpenAI;

var agent = Constant.GPT35.CreateAgent(
    name: "gpt35",
    roleInformation: "You are a helpful AI assistant",
    temperature: 0);

// create an AutoReplyAgent that don't want to talk
var autoReplyAgent = agent.CreateAutoReplyAgent(
    name: "autoReply",
    autoReplyMessageFunc: async (message, ct) => {
        // The autoReplyMessageFunc always return a message, therefore the inner agent will never be called.
        return new Message(Role.Assistant, "I don't want to talk to you", from: "autoReply");
    });

var agentReply = await agent.SendMessageAsync("hi");
agentReply.PrettyPrintMessage();

var autoReply = await autoReplyAgent.SendMessageAsync("hi");
autoReply.PrettyPrintMessage();

Message from gpt35
--------------------
Hello! How can I assist you today?
--------------------

Message from autoReply
--------------------
I don't want to talk to you
--------------------



### 2. use auto-reply to respond under certain circumstances

In most of the cases, we only want to use auto-reply function to generate a reply under certain circumstances. To achieve this, we can return 'null' from the auto-reply function to indicate that the auto-reply function does not generate a reply. In this case, the inner agent will be called next to generate a reply.

In [20]:
// create an AutoReplyAgent that don't want to talk if the message is longer than 1
var autoReplyAgent = agent.CreateAutoReplyAgent(
    name: "autoReply",
    autoReplyMessageFunc: async (message, ct) => {
        if (message.Count() > 1){
            return new Message(Role.Assistant, "I don't want to talk to you", from: "autoReply");
        }

        return null;
    });

// message count is 1, so the agent will reply
var agentReply = await autoReplyAgent.SendMessageAsync("hi");
agentReply.PrettyPrintMessage();

// message count is 2, the auto-reply function will return 'I don't want to talk to you' and the agent will not reply
var chatHistory = new [] {
    new Message(Role.User, "hi"),
    new Message(Role.User, "how's going"),
};
agentReply = await autoReplyAgent.SendMessageAsync(chatHistory);
agentReply.PrettyPrintMessage();

Message from gpt35
--------------------
Hello! How can I assist you today?
--------------------

Message from autoReply
--------------------
I don't want to talk to you
--------------------



### Scenario 1: I want to create an agent that can detect python and csharp code block. And reply 'no code block found' if no code block is detected.

It's difficult to use a single agent to deal with different situations. The first challenge is to write an appropriate system prompt that work for all the situations. The second challenge is the increase complexity of prompt might make the llm response less robust.

To address this issue, we can create multiple agents to deal with different situations. Then we can combine these agents into a single `AutoReplyAgent` to create a more general purpose agent. In the below example, we present how to use an `AutoReplyAgent` to combine two agents that can detect python and csharp code block respectively into a single agent.

In [22]:
// python detect agent only detect if the last message contains python code block
var pythonDetectAgent = Constant.GPT35.CreateAgent(
    "python",
    @"You are a helpful AI assistant, you detect if the last message contains python code block.
The python code block will be put between ```python and ```.
If the last message contains python code block, you will reply '[PYTHON CODE FOUND]'.
Otherwise, you will reply 'No python code found'",
    temperature: 0);

// test with pythonDetectAgent
var message = new Message(Role.User, "hi");
var reply = await pythonDetectAgent.SendMessageAsync(message);
reply.PrettyPrintMessage();

var pythonMessage = new Message(Role.User, @"
```python
print('hello world')
```");
reply = await pythonDetectAgent.SendMessageAsync(pythonMessage);
reply.PrettyPrintMessage();

Message from python
--------------------
No python code found
--------------------

Message from python
--------------------
[PYTHON CODE FOUND]
--------------------



In [24]:
// csharp detect agent only detect if the last message contains csharp code block
var csharpDetectAgent = Constant.GPT35.CreateAgent(
    "csharp",
    @"You are a helpful AI assistant, you detect if the last message contains csharp code block.
The python code block will be put between ```csharp and ```.
If the last message contains csharp code block, you will reply '[CSHARP CODE FOUND]'.
Otherwise, you will reply 'No csharp code found'",
    temperature: 0);

// test with csharpDetectAgent
var message = new Message(Role.User, "hi");
var reply = await csharpDetectAgent.SendMessageAsync(message);
reply.PrettyPrintMessage();

var pythonMessage = new Message(Role.User, @"
```csharp
Console.WriteLine(""hello world"")
```");
reply = await csharpDetectAgent.SendMessageAsync(pythonMessage);
reply.PrettyPrintMessage();

Message from csharp
--------------------
No csharp code found
--------------------

Message from csharp
--------------------
[CSHARP CODE FOUND]
--------------------



In [25]:
IAgent agent = Constant.GPT35.CreateAgent(
    name: "gpt35",
    roleInformation: "You reply 'No code found' in any case",
    temperature: 0);

// combine all the agents together to complete the code detection task.
var alice = agent.CreateAutoReplyAgent(
    name: "Alice",
    autoReplyMessageFunc: async (messages, ct) => {
        var lastMessage = messages.LastOrDefault();
        // first check if the last message contains python code block
        var pythonCodeBlockDetection = await pythonDetectAgent.SendMessageAsync(lastMessage);
        if (pythonCodeBlockDetection.Content.Contains("[PYTHON CODE FOUND]"))
        {
            return pythonCodeBlockDetection;
        }
        // then check if the last message contains csharp code block
        var csharpCodeBlockDetection = await csharpDetectAgent.SendMessageAsync(lastMessage);
        if (csharpCodeBlockDetection.Content.Contains("[CSHARP CODE FOUND]"))
        {
            return csharpCodeBlockDetection;
        }

        // let the agent reply No code found
        return null;
    });

In [27]:
var message = new Message(Role.User, "hi");
var reply = await alice.SendMessageAsync(message);
reply.PrettyPrintMessage();

var pythonMessage = new Message(Role.User, @"
```python
print('hello world')
```");
reply = await alice.SendMessageAsync(pythonMessage);
reply.PrettyPrintMessage();

var csharpMessage = new Message(Role.User, @"
```csharp
Console.WriteLine(""hello world"")
```");

reply = await alice.SendMessageAsync(csharpMessage);
reply.PrettyPrintMessage();

Message from gpt35
--------------------
No code found
--------------------

Message from python
--------------------
[PYTHON CODE FOUND]
--------------------

Message from csharp
--------------------
[CSHARP CODE FOUND]
--------------------

